In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
import os
import tqdm

import seaborn as sns
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision.utils import make_grid
from attrdict import AttrDict
import torch
import yaml
from sklearn.model_selection import StratifiedKFold
import copy
import pickle
# from tqdm import tqdm_notebook

# additional lightning 

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

import torch
from torch import nn
import torch.nn.functional as F
from timm import create_model

import gc

In [2]:
##################
# VM USE THIS
##################
import sys
sys.path.append("../")
from pawnet.utility import *


##################
# KAGGLE USE THIS
##################
# you need to import the utility script by uploading to kaggle as pawnet_utility/pawnet_utility.py
# from pawnet_utility import *

In [3]:
# extra augmentations for experiments
import pawnet.augmentations as aug_utils

In [4]:
# load config
# this object manages all the configurations
##################
# KAGGLE USE THIS
##################
# config_path = "../input/config/config.yaml"

##################
# VM USE THIS
##################
config_path = "../config/config.yaml"
base_config_manager = BaseConfigLoader(config_path)

In [5]:
##################
# KAGGLE USE THIS
##################
# this is specific to kaggle
# if running in GCS, replace with our GCP bucket
# get cache location of the dataset
# GCS_DS_PATH = KaggleDatasets().get_gcs_path()
# file_path = base_config_manager.load_config().filepath.kaggle #"/kaggle/input/petfinder-pawpularity-score/"


##################
# VM USE THIS
##################
file_path = base_config_manager.load_config().filepath.gcs

train_df = pd.read_csv(os.path.join(file_path,"train.csv"))
test_df = pd.read_csv(os.path.join(file_path,"test.csv"))

In [6]:
# (iterate model.named_children() instead of children())- https://medium.com/the-dl/how-to-use-pytorch-hooks-5041d777f904 
class featureExtractor:
    """
    Extracts feature maps/activate maps from various model
    """
    def get_activation_map(self,x_in, model,model_type="swin"):
        """
        add forward hook to the last output of swin transformer sequential block
        """
        
        # get last feature map from model.pretrained
        feature_map_mod = self.get_module_chidren(model=model,model_type=model_type)
        
        self.activation_value = None  # Stores the activation of the module you chose above during a forwards pass.
        def activation_hook(a, b, activation):
            self.activation_value = activation
        
        feature_map_mod.register_forward_hook(activation_hook)
        
        # forward
        model.eval()
        score = model(x_in)
        self.score = score
        
    def get_module_chidren(self,model,model_type="swin"):
        """
        Helper method to get last featuremap module depending on the architecture
        """
        
        # index depending on model:
        if model_type == "swin":
            last_feature_index = 3
        elif model_type == "resnet34":
            last_feature_index = 7
        elif model_type == "densenet":
            last_feature_index = 0
        elif model_type == "efficientnet":
            last_feature_index = 6
        elif model_type == "vit_small":
            last_feature_index = 2
        
        
        
        for i,param in enumerate(model.pretrained.children()):
            if i == last_feature_index:
                feature_map_mod = param
        
        return feature_map_mod    

# Load Data

In [7]:
model_config = base_config_manager.load_config().model.swin_tiny4_w7_224_cut1 # for general skf config
test_transformation = T.Compose([
                T.Resize([224,224]),# imgnet needs at least 224
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] ), # imgnet requirements 
                ]
            )

seed_everything(1)
skf = StratifiedKFold(n_splits = model_config.n_splits, shuffle = True, random_state = 1)
splits = skf.split(train_df["Id"],train_df["Pawpularity"])

rmse_list = []

# get data loader for fold 0
for i, (train_index, test_index) in enumerate(splits):
    print("\n Starting: fold {}".format(i+1))
    X_train, X_valid = train_df.iloc[train_index], train_df.iloc[test_index]
    X_train.reset_index(inplace=True,drop=True)
    X_valid.reset_index(inplace=True,drop=True)
    
    valid_data = pawnetDataset(annotation_df=X_valid,img_dir =os.path.join(file_path,"train"), transform = test_transformation,test=True)
    valid_loader = torch.utils.data.DataLoader(valid_data,batch_size=64,shuffle=False,num_workers=2)
    break

Global seed set to 1



 Starting: fold 1


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [8]:
# get first batch of data
x_1,y_1 = iter(valid_loader).next()

For the following sections, you need to run separately and not in 1 single run - else kernel will get error and die

# Load Swin

In [9]:
model_config = base_config_manager.load_config().model.swin_tiny4_w7_224_cut1
criterion = torch.nn.BCEWithLogitsLoss()
swin_model = pawNetBasic.load_from_checkpoint(checkpoint_path=f"pawnet_lightning_swin_tiny4_w7_224_mixup04/default/version_0/checkpoints/best_loss.ckpt",criterion=criterion,model_config=model_config)

will perform cutmix


In [10]:
fe = featureExtractor()

In [11]:
fe.get_activation_map(x_in=x_1,model=swin_model,model_type="swin")

In [12]:
fe.activation_value.shape

torch.Size([64, 49, 768])

In [13]:
swin_feature_map = fe.activation_value.clone()

In [14]:
del fe, swin_model
gc.collect()
torch.cuda.empty_cache()

In [16]:
with open("swin.pkl","wb") as fout:
    pickle.dump([swin_feature_map],fout)

# Load Densenet

In [9]:
model_config = base_config_manager.load_config().model.densenet121
criterion = torch.nn.BCEWithLogitsLoss()
densenet_model = pawNetBasic.load_from_checkpoint(checkpoint_path=f"pawnet_lightning_densenet/default/version_0/checkpoints/best_loss.ckpt",criterion=criterion,model_config=model_config)

In [10]:
dense_fe = featureExtractor()
dense_fe.get_activation_map(x_in=x_1,model=densenet_model,model_type="densenet")
denset_feature_map = dense_fe.activation_value.clone()

del dense_fe, densenet_model
gc.collect()
torch.cuda.empty_cache()

with open("densenet.pkl","wb") as fout:
    pickle.dump([denset_feature_map],fout)

# Load EfficientNet

In [12]:
model_config = base_config_manager.load_config().model.efficientnet_b0
criterion = torch.nn.BCEWithLogitsLoss()
effnet_model = pawNetBasic.load_from_checkpoint(checkpoint_path=f"pawnet_lightning_efficientnet_b0/default/version_0/checkpoints/best_loss.ckpt",criterion=criterion,model_config=model_config)

In [15]:
eff_fe = featureExtractor()
eff_fe.get_activation_map(x_in=x_1,model=effnet_model,model_type="efficientnet")
eff_feature_map = eff_fe.activation_value.clone()

del eff_fe, effnet_model
gc.collect()
torch.cuda.empty_cache()

with open("efficientnet.pkl","wb") as fout:
    pickle.dump([eff_feature_map],fout)

# Load Resnet

In [17]:
model_config = base_config_manager.load_config().model.resnet34
criterion = torch.nn.BCEWithLogitsLoss()
resnet_model = pawNetBasic.load_from_checkpoint(checkpoint_path=f"02a-resnet/pawnet_lightning_resnet/default/version_0/checkpoints/best_loss.ckpt",criterion=criterion,model_config=model_config)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet34-43635321.pth" to /home/jupyter/.cache/torch/hub/checkpoints/resnet34-43635321.pth


In [18]:
resnet_fe = featureExtractor()
resnet_fe.get_activation_map(x_in=x_1,model=resnet_model,model_type="resnet34")
resnet_feature_map = resnet_fe.activation_value.clone()

with open("resnet.pkl","wb") as fout:
    pickle.dump([resnet_feature_map],fout)

# load vit

In [9]:
model_config = base_config_manager.load_config().model.vit_tiny16_224
criterion = torch.nn.BCEWithLogitsLoss()
vit_model = pawNetBasic.load_from_checkpoint(checkpoint_path=f"02c-vit-small/pawnet_lightning_vit_tiny_patch16_224/default/version_0/checkpoints/best_loss.ckpt",criterion=criterion,model_config=model_config)

In [10]:
vit_fe = featureExtractor()
vit_fe.get_activation_map(x_in=x_1,model=vit_model,model_type="vit_small")
vit_feature_map = vit_fe.activation_value.clone()

with open("vit_small.pkl","wb") as fout:
    pickle.dump([vit_feature_map],fout)